In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import datetime

In [3]:
## source: https://github.com/ryendu/sound2ahap/blob/main/utils.py

class AHAP:
    def __init__(self):
        self.data = {
            "Version": 1.0,
            "Metadata": {
                "Project": "Basis",
                "Created": str(datetime.datetime.now()),
                "Description": "AHAP file generated based on a audio file by Basis app.",
                "Created By": "Isabella Gomez"
            },
            "Pattern": []
        }

    def add_event(self, etype, time, parameters, event_duration=None, event_waveform_path=None):
        """
        Adds an event to the pattern
        etype: type of event
            - possible values: AudioContinuous, AudioCustom, HapticTransient, and HapticContinuous
        time: time of event
            - in seconds
        parameters: event parameters
            - as a list of dictionaries
        """
        pattern = {
            "Event": {
                "Time": time,
                "EventType": etype,
                "EventParameters": parameters
            }
        }
        if event_duration != None:
            pattern["Event"]["EventDuration"] = event_duration
        if event_waveform_path != None:
            pattern["Event"]["EventWaveformPath"] = event_waveform_path
        self.data["Pattern"].append(pattern)

    def add_haptic_transient_event(self, time, haptic_intensity, haptic_sharpness):
        """
        Adds a haptic transient event to the pattern
        time: time of event - in seconds
        haptic_intensity: intensity of haptic
        haptic_sharpness: sharpness of haptic
        """
        parameters = [
            {
                "ParameterID": "HapticIntensity",
                "ParameterValue": haptic_intensity,
            },
            {
                "ParameterID": "HapticSharpness",
                "ParameterValue": haptic_sharpness,
            }
        ]

        self.add_event(etype="HapticTransient",
                       time=time, parameters=parameters)

    def add_haptic_continuous_event(self, time, haptic_intensity, haptic_sharpness, event_duration):
        """
        Adds a haptic continuous event to the pattern
        time: time of event - in seconds
        haptic_intensity: intensity of haptic
        haptic_sharpness: sharpness of haptic
        """
        parameters = [
            {
                "ParameterID": "HapticIntensity",
                "ParameterValue": haptic_intensity,
            },
            {
                "ParameterID": "HapticSharpness",
                "ParameterValue": haptic_sharpness,
            }
        ]

        self.add_event(etype="HapticContinuous", time=time,
                       parameters=parameters, event_duration=event_duration)

    def add_audio_custom_event(self, time, wav_filepath, volume=0.75):
        """
        Adds an audio custom event to the pattern
        time: time of event
            - in seconds
        wav_filepath: path to the wav file containing the sound
        volume: volume from 0 to 1
        """
        parameters = [
            {
                "ParameterID": "AudioVolume",
                "ParameterValue": volume,
            }
        ]
        self.add_event(etype="AudioCustom", time=time,
                       parameters=parameters, event_waveform_path=wav_filepath)

    def add_parameter_curve(self, parameter_id, start_time, control_points):
        """
        Adds a parameter curve to the pattern
        parameter_id: the parameter to dynamically change
            - possible values: HapticIntensityControl, HapticSharpnessControl, HapticAttackTimeControl, HapticDecayTimeControl, HapticReleaseTimeControl, AudioBrightnessControl, AudioPanControl, AudioPitchControl, AudioVolumeControl, AudioAttackTimeControl, AudioDecayTimeControl, AudioReleaseTimeControl
        start_time: time of the start of the curve
            - in seconds
        control_points: list of control points
            - as a list of dictionaries in the format: [{"Time":time,"ParameterValue":value}]
        """
        pattern = {
            "ParameterCurve": {
                "ParameterID": parameter_id,
                "Time": start_time,
                "ParameterCurveControlPoints": control_points
            }
        }

        self.data["Pattern"].append(pattern)

    def print_data(self):
        print(self.data)

    def export(self, filename, path):
        with open(os.path.join(path, filename), 'w') as f:
            f.write(json.dumps(self.data))

In [55]:
## open data
data_left = pd.read_csv('song1/choreo2_lleg.csv', names=['time', 'event'])

data_left.head()

,time,event
0,12.411,1
1,13.101,1
2,17.531,1
3,18.321,1
4,19.951,1


In [56]:
data_right = pd.read_csv('song1/choreo2_rleg.csv', names=['time', 'event'])

data_right.head()

,time,event
0,15.521,1
1,17.241,1
2,20.091,1
3,23.932,1
4,24.582,1


In [57]:
data = pd.concat([data_left, data_right], ignore_index=True)
data = data.sort_values('time', ignore_index=True)
data['time'] = data['time']-data['time'].iloc[0]+3

data.head()

,time,event
0,3.00,1
1,3.69,1
2,6.11,1
3,7.83,1
4,8.12,1


In [58]:
## write ahap

# heel = 0, stamp = 1, toe = 2, toe rev = 3, spin = 4
ahap = AHAP()

## add pulses at the start
ahap.add_haptic_transient_event(0, 0.99, 0.99)
ahap.add_haptic_transient_event(1, 0.99, 0.99)
ahap.add_haptic_transient_event(2, 0.99, 0.99)

funcs = [ahap.add_haptic_transient_event, ahap.add_haptic_continuous_event]
heel = {'event':0, 'sharp':0.05, 'int':0.85}
stamp = {'event':0, 'sharp':0.95, 'int':0.6}
toe = {'event':0, 'sharp':0.1, 'int':0.6}
toe_rev = {'event':0, 'sharp':0.8, 'int':0.5}
spin = {'event':1, 'sharp':0.15, 'int':0.55}

events = [heel, stamp, toe, toe_rev, spin]


for idx, event in data.iterrows():
    time = event['time']
    ev_type = event['event'].astype('int')
    fun = funcs[events[ev_type]['event']]
    sharp = events[ev_type]['sharp']
    int = events[ev_type]['int']

    fun(time, int, sharp)


In [44]:
## test

# funcs = [ahap.add_haptic_transient_event, ahap.add_haptic_continuous_event]
# heel = {'event':0, 'sharp':0.05, 'int':0.85}
# stamp = {'event':0, 'sharp':0.95, 'int':0.6}
# toe = {'event':0, 'sharp':0.1, 'int':0.6}
# toe_rev = {'event':0, 'sharp':0.8, 'int':0.5}
# spin = {'event':1, 'sharp':0.15, 'int':0.55}

# events = [heel, stamp, toe, toe_rev, spin]

# for idx, event in data.iterrows():
#     time = event['time']
#     print(time)
#     ev_type = event['event'].astype('int')
#     print(ev_type)
#     fun = funcs[events[ev_type]['event']]
#     print(fun)
#     sharp = events[ev_type]['sharp']
#     print(sharp)
#     int = events[ev_type]['int']
#     print(int)

8.671
0
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.05
0.85
10.541
0
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.05
0.85
11.151
1
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.95
0.6
12.441
0
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.05
0.85
12.521
1
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.95
0.6
13.651
1
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.95
0.6
14.361
1
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.95
0.6
14.761
1
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.95
0.6
14.801
0
<bound method AHAP.add_haptic_transient_event of <__main__.AHAP object at 0x1590b2dd0>>
0.05
0.85
15.261
0
<bound method AHAP.add_haptic_tran

## Simplified

In [14]:
data = pd.read_csv('cymbal_tracks/thillana_p2.csv', names=['time'])
data.head()

,time
0,12.076055
1,12.566078
2,13.106781
3,13.327529
4,13.594037


In [16]:
## adjust time
# data['time'] = data['time']+6
# data.head()

,time
0,23.263539
1,23.694673
2,25.553040
3,25.840274
4,26.161566


In [15]:
## write file - continuous
ahap = AHAP()

# ahap.add_haptic_transient_event(0, 0.99, 0.99)
# ahap.add_haptic_transient_event(1, 0.99, 0.99)
# ahap.add_haptic_transient_event(2, 0.99, 0.99)
# ahap.add_haptic_transient_event(3, 0.99, 0.99)
# ahap.add_haptic_transient_event(4, 0.99, 0.99)
# ahap.add_haptic_transient_event(5, 0.99, 0.99)
# ahap.add_haptic_transient_event(6, 0.99, 0.99)

for idx, event in data.iterrows():
    time = event['time']
    ahap.add_haptic_continuous_event(time, 0.99, 0.99, 0.02)


In [38]:
## write file - transient
ahap2 = AHAP()

ahap2.add_haptic_transient_event(0, 0.99, 0.99)
ahap2.add_haptic_transient_event(1, 0.99, 0.99)
ahap2.add_haptic_transient_event(2, 0.99, 0.99)
ahap2.add_haptic_transient_event(3, 0.99, 0.99)
ahap2.add_haptic_transient_event(4, 0.99, 0.99)
ahap2.add_haptic_transient_event(5, 0.99, 0.99)
ahap2.add_haptic_transient_event(6, 0.99, 0.99)

for idx, event in data.iterrows():
    time = event['time']
    ahap2.add_haptic_transient_event(time, 0.99, 0.99)

In [9]:
## print ahap
ahap.data

{'Version': 1.0,
 'Metadata': {'Project': 'Basis',
  'Created': '2023-11-15 09:16:23.452550',
  'Description': 'AHAP file generated based on a audio file by Basis app.',
  'Created By': 'Isabella Gomez'},
 'Pattern': [{'Event': {'Time': 0,
    'EventType': 'HapticTransient',
    'EventParameters': [{'ParameterID': 'HapticIntensity',
      'ParameterValue': 0.99},
     {'ParameterID': 'HapticSharpness', 'ParameterValue': 0.99}]}},
  {'Event': {'Time': 1,
    'EventType': 'HapticTransient',
    'EventParameters': [{'ParameterID': 'HapticIntensity',
      'ParameterValue': 0.99},
     {'ParameterID': 'HapticSharpness', 'ParameterValue': 0.99}]}},
  {'Event': {'Time': 2,
    'EventType': 'HapticTransient',
    'EventParameters': [{'ParameterID': 'HapticIntensity',
      'ParameterValue': 0.99},
     {'ParameterID': 'HapticSharpness', 'ParameterValue': 0.99}]}},
  {'Event': {'Time': 15.129048413366,
    'EventType': 'HapticContinuous',
    'EventParameters': [{'ParameterID': 'HapticIntensit

In [16]:
## export file

ahap.export(f'cymbal_tracks/thillana_p2.ahap','')
# ahap2.export(f'thillana/thillana2.ahap','')

## Demos

In [53]:
d = [5.6851, 6.3747, 7.0192, 7.6613, 8.3413, 9.0361, 9.6793, 10.3422]
d2 = [round(x-d[0], 4) for x in d]
print(d2)

[0.0, 0.6896, 1.3341, 1.9762, 2.6562, 3.351, 3.9942, 4.6571]


In [46]:
d2 = [round(x + 3.486 + (3.486-3.0295), 4) for x in d2]
print(d2)

[3.9425, 4.1889, 4.4061, 4.6083, 4.8936, 5.1358, 5.344, 5.5774, 5.8297, 6.0772, 6.2903, 6.5098, 6.764, 7.0043, 7.2316, 7.458]


In [61]:
## write ahap
ahap_d = AHAP()

## demo 1: 0.0, 0.6016, 1.1734, 1.7899, 2.3947, 3.0079, 3.5927, 3.8596, 4.1451, 4.4286, 4.6953, 4.9924, 5.2896, 5.5598, 5.8525, 6.1363, 6.3902, 6.6766
# ahap_d.add_haptic_transient_event(0.0, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(0.6016, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(1.1734, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(1.7899, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(2.3947, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.0079, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.5927, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.8596, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.1451, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.4286, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.6953, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.9924, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.2896, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.5598, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.8525, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.1363, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.3902, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.6766, 0.99, 0.99)


## demo 2: 0.0, 0.4727, 1.0213, 1.5063, 2.048, 2.5361, 3.0295, 3.486, 3.9425, 4.1889, 4.4061, 4.6083, 4.8936, 5.1358, 5.344, 5.5774, 5.8297, 6.0772, 6.2903, 6.5098, 6.764, 7.0043, 7.2316, 7.458
# ahap_d.add_haptic_transient_event(0.0, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(0.4727, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(1.0213, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(1.5063, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(2.048, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(2.5361, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.0295, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.486, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.9425, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.1889, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.4061, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.6083, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(4.8936, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.1358, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.344, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.5774, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(5.8297, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.0772, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.2903, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.5098, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(6.764, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(7.0043, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(7.2316, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(7.458, 0.99, 0.99)


## demo 3: 0.0, 0.3359, 0.6372, 0.9474, 1.2521, 1.5656, 1.8706, 2.1728

## demo 5: 0.0, 0.6896, 1.3341, 1.9762, 2.6562, 3.351, 3.9942, 4.6571
ahap_d.add_haptic_continuous_event(0.0, 0.99, 0.99, 0.1)
ahap_d.add_haptic_transient_event(0.6896, 0.99, 0.99)
ahap_d.add_haptic_transient_event(1.3341, 0.99, 0.99)
ahap_d.add_haptic_transient_event(1.9762, 0.99, 0.99)
ahap_d.add_haptic_continuous_event(2.6562, 0.99, 0.99, 0.1)
ahap_d.add_haptic_transient_event(3.351, 0.99, 0.99)
ahap_d.add_haptic_transient_event(3.9942, 0.99, 0.99)
ahap_d.add_haptic_transient_event(4.6571, 0.99, 0.99)

## demo 6: 0.0, 0.2829, 1.1099, 1.8564, 3.3617, 4.0116, 4.3154, 4.6455, 4.9327, 5.2904, 5.9425, 6.5925, 7.2797, 8.1204, 8.6918, 9.2634, 9.5811, 9.8692
# ahap_d.add_haptic_continuous_event(0.0, 0.99, 0.6, 0.651)
# ahap_d.add_haptic_transient_event(0.6521, 0.99, 0.99)
# ahap_d.add_haptic_continuous_event(1.3021, 0.99, 0.6, 0.6861)
# ahap_d.add_haptic_transient_event(1.9893, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(2.83, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(3.4014, 0.99, 0.99)
# ahap_d.add_haptic_continuous_event(3.973, 0.99, 0.99, 0.02)
# ahap_d.add_haptic_continuous_event(4.2907, 0.99, 0.99, 0.02)
# ahap_d.add_haptic_continuous_event(4.5788, 0.99, 0.99, 0.02)

# ahap_d.add_haptic_continuous_event(5.2904, 0.99, 0.6, 0.651)
# ahap_d.add_haptic_transient_event(5.9425, 0.99, 0.99)
# ahap_d.add_haptic_continuous_event(6.5925, 0.99, 0.6, 0.6861)
# ahap_d.add_haptic_transient_event(7.2797, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(8.1204, 0.99, 0.99)
# ahap_d.add_haptic_transient_event(8.6918, 0.99, 0.99)
# ahap_d.add_haptic_continuous_event(9.2634, 0.99, 0.99, 0.02)
# ahap_d.add_haptic_continuous_event(9.5811, 0.99, 0.99, 0.02)
# ahap_d.add_haptic_continuous_event(9.8692, 0.99, 0.99, 0.02)



In [62]:
## export
ahap_d.export(f'demos/demo5_mixed.ahap','')